In [76]:
import numpy as np
import matplotlib.pyplot as plt

In [77]:
# Input:

Den_h2so4 = 1.8 # g/cm³

tmp = [220,240,260] # temperature [K]
tmp = np.array(tmp)

aw = [0.5, 0.7, 0.9, 0.99] # = relative humidity / 100

Rdry = [0.089, 1.89, 10.0] # [um]
Rdry = np.array(Rdry)

In [93]:
# !        calculation of particle density
# !        requires Temperature (temp) and acid mass fraction (so4mfrac)
# !
# !
# !---->Vehkamaeki et al., 2002  (JGR, doi:10.1029/2002JD002184)

def density(temp,so4mfrac):
    
    a1= 0.7681724
    a2= 2.184714
    a3= 7.163002
    a4=-44.31447
    a5= 88.74606
    a6=-75.73729
    a7= 23.43228
    
    b1= 1.808225e-3
    b2=-9.294656e-3
    b3=-3.742148e-2
    b4= 2.565321e-1
    b5=-5.362872e-1
    b6= 4.857736e-1
    b7=-1.629592e-1
     
    c1=-3.478524e-6
    c2= 1.335867e-5
    c3= 5.195706e-5
    c4=-3.717636e-4
    c5= 7.990811e-4
    c6=-7.458060e-4
    c7= 2.581390e-4
    
 
    so4m2=so4mfrac*so4mfrac
    so4m3=so4mfrac*so4m2
    so4m4=so4mfrac*so4m3
    so4m5=so4mfrac*so4m4
    so4m6=so4mfrac*so4m5

    a=a1+a2*so4mfrac+a3*so4m2+a4*so4m3 +a5*so4m4+a6*so4m5+a7*so4m6
    b=b1+b2*so4mfrac+b3*so4m2+b4*so4m3 +b5*so4m4+b6*so4m5+b7*so4m6
    c=c1+c2*so4mfrac+c3*so4m2+c4*so4m3 +c5*so4m4+c6*so4m5+c7*so4m6
      
    density=(a+b*temp+c*temp*temp) # units are g/cm**3
        
    return density

In [103]:

Rwet = np.zeros((len(tmp), len(aw), len(Rdry)))

for i in range(len(tmp)):
    for k in range(len(aw)):
        
        if aw[i]==0.0:
            print("The wet radius is same as the dry radius")
            print("Just stop and no need to run the rest!")
    
    
        if aw[k]>1.0: aw[k]=1.0
        if aw[k]<=0.05:
            y1=12.372089320*aw[k]**(-0.16125516114) -30.490657554*aw[k] -2.1133114241
            y2=13.455394705*aw[k]**(-0.19213122550) -34.285174607*aw[k] -1.7620073078
        elif aw[k]<=0.85:
            y1=11.820654354*aw[k]**(-0.20786404244) -4.8073063730*aw[k] -5.1727540348
            y2=12.891938068*aw[k]**(-0.23233847708) -6.4261237757*aw[k] -4.9005471319
        else:
            y1=-180.06541028*aw[k]**(-0.38601102592) -93.317846778*aw[k] +273.88132245
            y2=-176.95814097*aw[k]**(-0.36257048154) -90.469744201*aw[k] +267.45509988

        sulfmolal=y1+((tmp[i]-190.)*(y2-y1)/70.)
        
        wp=9800.*sulfmolal/(98.*sulfmolal+1000.) # weight percentage
        
        if wp<15.0: wp=15.0
        if wp>99.0: wp=99.0
        if wp<15.0 or wp>100.0:
            print( 'weight percent= ',wp,  ', T=',tmp[i], ', RH(aw)=', aw[k])
            print('*** Stupid Value of Weight Percent ***')
             
                
        # mass fraction
        wpp=wp*0.01
        # mole fraction
        xa=18.*wpp/(18.*wpp+98.*(1.-wpp))
        
        
        den = density(tmp[i],wpp)
        
        
        Vdry = 4/3 *np.pi * Rdry[:]**3
        Mdry = Den_h2so4*Vdry[:]

        Mwet = Mdry[:]/wpp
        Vwet = Mwet[:]/den
        
        Rwet[i,k,:] = (Vwet[:] / (4/3 *np.pi))**(1/3)
        
        print( f'temp={tmp[i]:.1f}K, RH={aw[k]*100:.1f}%, wp={wp:.3f} %, xa={xa:.3f}, den={den:.3f} gm/cm**3' )
        print(f'Rwet={Rwet[i,k,:]}')
        print('')
    
    

temp=220.0K, RH=50.0%, wp=38.859 %, xa=0.105, den=1.348 gm/cm**3
Rwet=[ 0.13428893  2.85175361 15.08864343]

temp=220.0K, RH=70.0%, wp=29.985 %, xa=0.073, den=1.269 gm/cm**3
Rwet=[ 0.14938642  3.17236321 16.78499053]

temp=220.0K, RH=90.0%, wp=18.275 %, xa=0.039, den=1.163 gm/cm**3
Rwet=[ 0.18139704  3.85213933 20.38168957]

temp=220.0K, RH=99.0%, wp=15.000 %, xa=0.031, den=1.133 gm/cm**3
Rwet=[ 0.19547001  4.15099237 21.96292261]

temp=240.0K, RH=50.0%, wp=39.843 %, xa=0.108, den=1.342 gm/cm**3
Rwet=[ 0.1333957   2.83278511 14.98828099]

temp=240.0K, RH=70.0%, wp=30.550 %, xa=0.075, den=1.260 gm/cm**3
Rwet=[ 0.14881573  3.16024421 16.72086886]

temp=240.0K, RH=90.0%, wp=17.952 %, xa=0.039, den=1.151 gm/cm**3
Rwet=[ 0.18310899  3.88849439 20.57404437]

temp=240.0K, RH=99.0%, wp=15.000 %, xa=0.031, den=1.126 gm/cm**3
Rwet=[ 0.19588017  4.15970241 22.00900744]

temp=260.0K, RH=50.0%, wp=40.795 %, xa=0.112, den=1.335 gm/cm**3
Rwet=[ 0.13257729  2.81540532 14.89632446]

temp=260.0K, RH=70.

In [106]:
# Rwet's sensitivity to temp
# Rwet = np.zeros((len(tmp), len(aw), len(Rdry)))

Diff = np.zeros(( len(aw), len(Rdry) ))

for j in range(len(aw)):
    for k in range(len(Rdry)):
        Diff[j,k] = (np.max(Rwet[:,j,k])-np.min(Rwet[:,j,k]))/np.min(Rwet[:,j,k])

In [108]:
Diff*100

array([[1.29104978, 1.29104978, 1.29104978],
       [0.7375243 , 0.7375243 , 0.7375243 ],
       [1.92663714, 1.92663714, 1.92663714],
       [0.45040853, 0.45040853, 0.45040853]])

## Conclusion: the change of wet radius caused by temperature is less than 2%, which indicates we can neglect temp's influence on wet radius in the stratosphere.